In [24]:
import pandas as pd 

## Load and Process the File

In [25]:
import pandas as pd 
from tqdm import tqdm 
from langchain.docstore.document import Document
import chardet 

def creare_doc_list_from_excel(file_path):
    with open(file_path, 'rb') as file :
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    df = pd.read_csv(file_path, encoding=encoding, delimiter=',')
    df = df.drop_duplicates( )

    
    
    if df.empty:
        print(" File has no data !!! ")
        return None
    doc_list = doc_creator(df)
    return df, doc_list


def doc_creator(df):
    doc_list= []
    avail_columns = df.columns
    
    
    for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing Rows'):
        product_string = ""
        produnct_metadata = {}
        for  col in avail_columns:
            value =row[col]
            product_string += f"{col}: {value}\n"
            
            if 'name' in col.lower():

                produnct_metadata["Name"] = value

            elif 'category' in col.lower():

                produnct_metadata["Categories"] = value

        doc = Document(page_content=product_string, metadata = produnct_metadata)

        doc_list.append(doc)

    return doc_list

# Main code: Provide the file path

file_path = r"insurance_loan_services_enriched.csv" 

#Call the function to load the file and process it

df, docs = creare_doc_list_from_excel(file_path)

if docs is None:
    print("Failed to load the documents.")
else:
    print("Loaded (len(docs)) documents.")


Processing Rows: 100%|██████████| 6/6 [00:00<00:00, 240.91it/s]

Loaded (len(docs)) documents.


In [27]:
docs

[Document(metadata={'Name': 'Home Loan'}, page_content='Service Name: Home Loan\nEligibility Criteria: Age 25-60, Income 50000+\nInterest Rate/Premium: 8% p.a.\nApplication Process: Apply online, provide ID, proof of income, property documents\nContact Information: 123-4567890, support@company.com\nPolicy Terms: Payback in 15 years, EMI starts after 2 months\nService Locations: Delhi, Mumbai, Bangalore\nLoan/Insurance Type: Secured Loan\nApproval Time: 7-10 business days\nSuggested For: Homebuyers, Families\nHow To Apply: Apply online, submit documents, get approval\nEligibility Criteria For: People with stable income\nReturn Amount/Benefit: 1.2X loan value\n'),
 Document(metadata={'Name': 'Personal Loan'}, page_content='Service Name: Personal Loan\nEligibility Criteria: Age 23-55, Income 40000+\nInterest Rate/Premium: 10% p.a.\nApplication Process: Apply online, provide ID, bank statements\nContact Information: 123-4567891, help@company.com\nPolicy Terms: Payback in 5 years, EMI start

## Split Documents Using Text Splitter

In [28]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
documents

[Document(metadata={'Name': 'Home Loan'}, page_content='Service Name: Home Loan\nEligibility Criteria: Age 25-60, Income 50000+\nInterest Rate/Premium: 8% p.a.\nApplication Process: Apply online, provide ID, proof of income, property documents\nContact Information: 123-4567890, support@company.com\nPolicy Terms: Payback in 15 years, EMI starts after 2 months\nService Locations: Delhi, Mumbai, Bangalore\nLoan/Insurance Type: Secured Loan\nApproval Time: 7-10 business days\nSuggested For: Homebuyers, Families\nHow To Apply: Apply online, submit documents, get approval\nEligibility Criteria For: People with stable income\nReturn Amount/Benefit: 1.2X loan value'),
 Document(metadata={'Name': 'Personal Loan'}, page_content='Service Name: Personal Loan\nEligibility Criteria: Age 23-55, Income 40000+\nInterest Rate/Premium: 10% p.a.\nApplication Process: Apply online, provide ID, bank statements\nContact Information: 123-4567891, help@company.com\nPolicy Terms: Payback in 5 years, EMI starts 

## Create FAISS Vector Database

In [32]:
# from langchain_community.vectorstores import FAISS
# data embedding and VEctorstore 

from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents, OllamaEmbeddings())
retriver = vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000011507292390>, search_kwargs={})

## Create the Retriever Tool

In [33]:
from langchain.tools.retriever import create_retriever_tool

retrival_tool = create_retriever_tool(retriver, 'insurance_tool', "this is tool for Insurance Chatbot")
retrival_tool.name

'insurance_tool'

In [34]:
tools = [retrival_tool]

## Set Up the Chat Model

In [35]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model='llama2')

C:\Users\sopan\AppData\Local\Temp\ipykernel_19712\845362186.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model='llama2')


## Define the System Chat Prompt Template

In [1]:
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("""
You are an intelligent insurance assistant providing concise, pointwise responses based on a knowledge base of various insurance and loan services. Your responses should be:

1. **Brief and Clear**: Provide concise, clear, and actionable information.
2. **Professional and Helpful**: Maintain a professional tone and focus on answering user queries.
3. **Contextual Relevance**: Refer to the context provided to deliver the most relevant and accurate answers based on the insurance/loan service data.
4. **Pointwise Structure**: Break down your answers into easy-to-understand, pointwise information.
5. **Use the Vector Database**: Utilize the provided vector database to search for relevant information based on the user’s query.

<context>
{context}
</context>
Question: {input}
{agent_scratchpad}
""")



## Create OpenAI Tools Agent

In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

## Create Agent Executor

In [44]:
from langchain.agents import AgentExecutor

agent_axecuter = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_axecuter

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'context', 'input'], input_types={}, partial_variables={}, template="\nYou are an insurance assistant providing concise, pointwise answers. Your responses should be:\n1. Brief, clear, and to the point.\n2. Professional and helpful.\n3. Relevant to the user's query.\n\n<context>\n{context}\n</context>\nQuestion: {input}\n{agent_scratchpad}\n"), additional_kwargs={})])
| RunnableBinding(bound=ChatOllama(), kwargs={'tools': [{'type': 'function', 'function': {'name': 'insurance_tool', 'description': 'this is tool for Insurance Chatbot', 'parameters': {'properties': {'query': {'descriptio

####  Process User Query

In [47]:
input_query = "How do I apply for a Personal Loan?"


In [48]:
response = agent_axecuter.invoke({'context': retriver, 'input': input_query})
response



> Entering new AgentExecutor chain...
To apply for a personal loan, follow these steps:

1. Check your credit score and history to ensure you meet the lender's eligibility criteria.
2. Research and compare interest rates and terms offered by different lenders.
3. Prepare your financial documents, such as income statements, bank statements, and tax returns.
4. Choose a reputable lender and submit your application online or through their branch.
5. Wait for the lender to process your application and make a decision.

Remember to compare interest rates, terms, and fees before making a decision. It's also important to understand the repayment schedule and any potential penalties for late payments.

> Finished chain.


{'context': VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000011507292390>, search_kwargs={}),
 'input': 'How do I apply for a Personal Loan?',
 'output': "To apply for a personal loan, follow these steps:\n\n1. Check your credit score and history to ensure you meet the lender's eligibility criteria.\n2. Research and compare interest rates and terms offered by different lenders.\n3. Prepare your financial documents, such as income statements, bank statements, and tax returns.\n4. Choose a reputable lender and submit your application online or through their branch.\n5. Wait for the lender to process your application and make a decision.\n\nRemember to compare interest rates, terms, and fees before making a decision. It's also important to understand the repayment schedule and any potential penalties for late payments."}

## Save Objects as Pickle Files

In [46]:
import pickle

# Save the VectorDB
with open("vectordb.pkl", "wb") as vectordb_file:
    pickle.dump(vectordb, vectordb_file)

# Save the tools
with open("tools.pkl", "wb") as tools_file:
    pickle.dump(tools, tools_file)

# Save the LLM model
with open("llm_model.pkl", "wb") as llm_file:
    pickle.dump(llm, llm_file)

# Save the prompt
with open("prompt.pkl", "wb") as prompt_file:
    pickle.dump(prompt, prompt_file)

print("All objects have been successfully saved as pickle files.")


All objects have been successfully saved as pickle files.
